# Laboratorio 4 - Parte 1

### Redes Neuronales Artificiales: MLP

### 2018-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: John Alexander Galeano Ospina
#### Segundo Integrante: Sidney Paola Aguirre Castro

In [77]:
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt
from __future__ import division

#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")

## Indicaciones

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de regresión. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

Para este ejercicio usaremos la base de datos sobre calidad del aire, que ha sido usada en laboratorios previos, pero en este caso trataremos de predecir dos variables en lugar de una, es decir, abordaremos un problema de múltiples salidas.

In [78]:
#cargamos la bd que está en un archivo .data y ahora la podemos manejar de forma matricial
db = np.loadtxt('DB/AirQuality.data',delimiter='\t')  # Assuming tab-delimiter

#Esta es la base de datos AirQuality del UCI Machine Learning Repository. En la siguiente URL se encuentra toda
#la descripción de la base de datos y la contextualización del problema.
#https://archive.ics.uci.edu/ml/datasets/Air+Quality#

X = db[:,0:11]
Y = db[:,11:13]

In [79]:
np.unique(Y)

array([-2.000e+02,  1.847e-01,  1.862e-01, ...,  8.710e+01,  8.720e+01,
        8.870e+01])

In [80]:
#Mean Absolute Percentage Error para los problemas de regresión
def MAPE(Y_est,Y):
    N = np.size(Y)
    mape = np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))/N
    return mape 

## Ejercicio 1

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de regresión propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [91]:
from numpy import random
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing
#Validamos el modelo
Folds = 4
random.seed(19680801)
ErrorY1 = np.zeros(Folds)
ErrorY2 = np.zeros(Folds)
ErrorT = np.zeros(Folds)
ss = ShuffleSplit(n_splits=Folds, test_size=0.3)
j = 0
for train, test in ss.split(X):
    Xtrain = X[train,:]
    Ytrain = Y[train,:]
    Xtest = X[test,:]
    Ytest = Y[test,:]
    
    #Normalizamos los datos
    media = np.mean(Xtrain,axis=0)
    desvia = np.std(Xtrain,axis=0)
    Xtrain = preprocessing.scale(Xtrain)
    Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    epochs = 500
    neurons = 36
    mlp = MLPRegressor(hidden_layer_sizes=(neurons,neurons ),activation='tanh', max_iter=epochs)
    mlp.fit(Xtrain,Ytrain)
    fnO = mlp.activation
    
    #Use para el modelo para hacer predicciones sobre el conjunto Xtest
    Yest = mlp.predict(Xtest)
    
    #Mida el error MAPE para cada una de las dos salidas
    ErrorY1[j] = MAPE(Yest[:,0], Ytest[:,0])
    ErrorY2[j] = MAPE(Yest[:,1], Ytest[:,1])
    ErrorT[j] = (ErrorY1[j] + ErrorY2[j])/2
    j += 1

print(fnO)
#print(Yest.shape)  
print('MAPE salida 1 = ' + str(np.mean(ErrorY1)) + '+-' + str(np.std(ErrorY1)))
print('MAPE salida 2 = ' + str(np.mean(ErrorY2)) + '+-' + str(np.std(ErrorY2)))
print('MAPE total = '    + str(np.mean(ErrorT)) + '+-' + str(np.std(ErrorT)))

tanh
MAPE salida 1 = 0.06600835243363894+-0.0016428393530221281
MAPE salida 2 = 1.0220498948011731+-0.07211487660609449
MAPE total = 0.5440291236174061+-0.03573485899945333


In [19]:
Yest[:,0].reshape(2808,1).shape
Y.shape

(9357, 2)

N. de capas ocultas	  Neuronas por capa	       MAPE salida 1		 MAPE salida 2
    1                      20                    0.0862              2.3985
    1                      24                    0.0820              2.0704                
    1                      28                    0.0772              1.7329        
    1                      32                    0.0769              1.5158
    1                      36                    0.0756              1.4609
    
    2                      20                    0.0714              1.2207                       
    2                      24                    0.0662              1.1099
    2                      28                    0.0678              1.0028
    2                      32                    0.0672              1.0943
    2                      36                    0.0660              1.0220

## Ejercicio 2

Una vez completado el código anterior. Realice los experimentos necesarios para completar la tabla siguiente:

In [49]:
import pandas as pd
import qgrid
qgrid.nbinstall(overwrite=True)
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types["MAPE salida 1"] = ""
df_types["IC MAPE salida 1"] = ""
df_types["MAPE salida 2"] = ""
df_types["IC MAPE salida 2"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

ModuleNotFoundError: No module named 'qgrid'

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

qgrid_widget.get_changed_df()

<b>Responda</b>:

2.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 

R: En la capa de salida, el modelo no usa función de activación, ya que es una regresión, y la salida es un valor continuo.

## Ejercicio 3.

A continuación se leen los datos de un problema de clasificación. El problema corresponde a la clasifiación de dígitos escritos a mano. Usaremos únicamente 4 de las 10 clases disponibles. Los datos fueron preprocesados para reducir el número de características. La técnica usada será analizada más adelante en el curso.

In [54]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de clasificación. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de clasificación propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [67]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
j = 0
for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    
    #Normalizamos los datos
    media = np.mean(Xtrain)
    desvia = np.std(Xtrain)
    Xtrain = preprocessing.scale(Xtrain)
    Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    epochs = 500
    neurons = 36
    mlp = MLPClassifier(hidden_layer_sizes=(neurons,neurons ), activation='tanh', max_iter=epochs)
    mlp.fit(Xtrain,Ytrain)
    fnO = mlp.out_activation_ 
    nO = mlp.n_outputs_ 
    
    #Validación con las muestras de entrenamiento
    Ytrain_pred =  mlp.predict(Xtrain)
     
    #Validación con las muestras de test    
    Yest = mlp.predict(Xtest)
    
    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain)
    EficienciaVal[j] = np.mean(Yest == Ytest)
    j += 1
print(fnO)  
print(nO)
print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))

softmax
4
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9610542147143794+-0.01970756737185292


In [ ]:
N. de capas ocultas	  Neuronas por capa	 Eficiencia en validacion	 Intervalo de confianza
    1                      20                0.9637                      0.0267
    1                      24                0.9623                      0.0369
    1                      28                0.9637                      0.0246
    1                      32                0.9596                      0.0293
    1                      36                0.9553                      0.0339
    
    2                      20                0.9693                      0.0225
    2                      24                0.9526                      0.0374
    2                      28                0.9512                      0.0337
    2                      32                0.9568                      0.0349
    2                      36                0.9610                      0.0197

## Ejercicio 4

Una vez completado el código realice los experimentos necesarios para llenar la siguiente tabla:

In [48]:
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

NameError: name 'qgrid' is not defined

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [47]:
qgrid_widget.get_changed_df()

NameError: name 'qgrid_widget' is not defined

<b>Responda</b>:

4.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 
R: softmax

4.2 ¿Cuántas neuronas en la capa de salida tiene el modelo?¿Porqué debe tener ese número?
R: 4 neuronas en la capa de salida. Debe tener ese número porque el problema tiene 4 clases en las que se clasifica cada muestra